# CISC 452 Assignment 3 - Unsupervised Learning (100 points)  

Please put your name and student id

    Jonah Zimmer, #20123084

- The notebook file has clearly marked blocks where you are expected to write code. Do not write or modify any code outside of these blocks.
- Do not add or delete cells from the notebook.
- Run all cells, and do not clear out the outputs, before submitting. You will only get credit for code that has been run.
- Make sure to run all the cells from beginning before the submission
- Mark will be deducted based on late policy (-10% per day after due date until the end date after which no assignments will be accepted)

## [Part 1 Simple Competitive Learning (50 points)](#Part-1-Simple-Competitive-Learning)  

Build a **Kohonen Network** and use **Maxnet** at the output layer to find the node with the highest activation.  

### Build Model (30 points)  
Kohonen Network (20 points)  
Maxnet (10 points)  

### Evaluate Model (20 points)  
Use the Kohonen Network to predict the lables with the train and test datasets (10 points)  
Evaluate the prediction results (10 points)  
- Evaluation matrics include confusion matrix and accuracy

## [Part 2 Principle Component Analysis (50 points)](#Part-2-Principle-Component-Analysis)

Implement a **PCA Network** (not PCA) to reduce the dimension of the Iris dataset from 4 to 3.  
Use the Kohonen Network in Part 1 to train and test on the new dataset.  

### Build Model (30 points)  
Build PCA Network (20 points)  
Train the PCA model and obtain the new datasets with reduced dimension (10 points)  

### Evaluate Model (20 points)  
Use the Kohonen Network to predict the lables with the new train and test datasets (10 points)  
Evaluate the prediction results (10 points)  
- Evaluation matrics include confusion matrix and accuracy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

In [2]:
data = load_iris()
x = data.data
x = (x - x.mean(axis=0)) / x.std(axis=0)
y = data.target
data.feature_names, data.target_names

(['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 array(['setosa', 'versicolor', 'virginica'], dtype='<U10'))

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3)
x_train.shape, x_test.shape, y_train.shape, y_test.shape


((120, 4), (30, 4), (120,), (30,))

In [4]:
def evaluator(y_test, y_pred):
    ####################################################################################################
    # implement the evaluation matrices including confusion matrix and accuracy

    true_pos=0
    false_pos=0
    true_neg=0
    false_neg = 0
    for idx,test in enumerate(y_test):
        
        if test == y_pred[idx]:
            if y_pred[idx] == 1 or y_pred[idx] == 2:
                true_pos += 1
            else:
                true_neg += 1
        else:
            if y_pred[idx] ==1 or y_pred[idx] == 2:
                false_pos += 1
            else:
                false_neg += 1
    confusion_matrix = [[true_neg, false_pos],[false_neg, true_pos]]
    accuracy = (true_pos + true_neg)/ (true_pos + true_neg + false_pos + false_neg)
    precision = true_pos/(true_pos + false_pos)
    recall = true_pos/(true_pos + false_pos)
    f1 = 2*((precision*recall)/(precision + recall))
    print("Confusion Matrix")
    print(confusion_matrix[0])
    print(confusion_matrix[1])
    print("Accuracy: "+ str(accuracy))
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("F1 Score: " + str(f1))
    ####################################################################################################

In [5]:
# baseline model with K-Means
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3) #n_clusters - the number of clusters
km.fit(x_train)
y_pred = km.predict(x_train)
evaluator(y_train, y_pred)
y_pred = km.predict(x_test)
evaluator(y_test, y_pred)

Confusion Matrix
[0, 54]
[34, 32]
Accuracy: 0.26666666666666666
Precision: 0.37209302325581395
Recall: 0.37209302325581395
F1 Score: 0.3720930232558139
Confusion Matrix
[0, 13]
[10, 7]
Accuracy: 0.23333333333333334
Precision: 0.35
Recall: 0.35
F1 Score: 0.35


## Part 1 Simple Competitive Learning

In [6]:
####################################################################################################
#  build the Kohonen Network with Maxnet
class Kohonen(object):
    def __init__(self, centroid_num):
        self.centroid_num = centroid_num
        self.history = {}
        self.history['train_acc'], self.history['val_acc'], self.history['loss'] = [], [], []
    
    def maxNet(self,x):
        return np.argmax(x)

    def train(self, x,y, x_val, y_val, learning_rate=0.01, n_iters=100 ):
        input_size = x.shape
        high = np.max(x)
        low = np.min(x)
        self.W = [(np.random.randint(low,high,input_size)).astype(np.float32),
                  (np.random.randint(low,high,input_size)).astype(np.float32),
                  (np.random.randint(low,high,input_size)).astype(np.float32)]
        
        
        for i in range(n_iters):
            for xi in x:
                distances= np.zeros(self.centroid_num)
                for i in range(self.centroid_num):
                    distances[i] = sum(np.dot(self.W[i],xi))
            
                closest = self.maxNet(distances)
                
                weight_change = learning_rate*(xi-self.W[closest])
                self.W[closest] += weight_change
        clusters = np.zeros([3, self.centroid_num])
        
        index = 0
        for xi in x:
            distances= np.zeros(self.centroid_num)
            for i in range(self.centroid_num):
                distances[i] = sum(np.dot(self.W[i],xi))
            actual= y[index]
            cluster =self.maxNet(distances)
            
            clusters[cluster, actual] += 1
            index += 1
        
        self.outputs_cluster = np.empty(3)
        for i in range(3):
            max = 0
            for j in range(3):
                if clusters[i,j] > max:
                    max = clusters[i,j]
                    index = j
            self.outputs_cluster[i] = index
        
                

    def predict(self, x):
        y_pred = []
        for xi in x:
            distances= np.zeros(self.centroid_num)
            for i in range(self.centroid_num):
                distances[i] = sum(np.dot(self.W[i],xi))
            
            closest = self.maxNet(distances)
            y_pred.append( int(self.outputs_cluster[closest]))
        
            
        
        return y_pred


####################################################################################################

In [7]:
####################################################################################################
# test the Kohonen Network

model = Kohonen(3)
model.train(x_train,y_train,x_test,y_test)
prediction =model.predict(x_test)
evaluator(y_test,prediction)
####################################################################################################

Confusion Matrix
[10, 5]
[0, 15]
Accuracy: 0.8333333333333334
Precision: 0.75
Recall: 0.75
F1 Score: 0.75


## Part 2 Principle Component Analysis

In [8]:
####################################################################################################
# build the PCA model
class PCA(object):
    def __init__(self, n_comp):
        self.n_comp = n_comp
        self.w = None
        self.mean =None 

    def train(self, x):
        self.mean = np.mean(x, axis=0)
        x = x-self.mean
        
        cov = np.cov(x.T)
        
        eigen_values, eigen_vector = np.linalg.eig(cov)
        eigen_vector = eigen_vector.T
        indxs = np.argsort(eigen_values)
        indxs = np.flip(indxs)
        eigen_values = eigen_values[indxs]
        eigen_vector = eigen_vector[indxs]
        
        self.w = eigen_vector[0:self.n_comp]
        
        
        
        
        
    def predict(self, x):
        x = x-self.mean
        return np.dot(x,self.w.T)


####################################################################################################

In [9]:
####################################################################################################
#  train the PCA model and obtain the new train and test datasets
pca=PCA(3)
pca.train(x_train)
x_train_PCA=pca.predict(x_train)
x_test_PCA=pca.predict(x_test)
print('Final weight is:\n', pca.w)

####################################################################################################

Final weight is:
 [[ 0.50461723 -0.27322752  0.58719189  0.57088866]
 [-0.37423748 -0.92393427 -0.04440896 -0.06572386]
 [-0.7273616   0.24377541  0.1272058   0.62875856]]


In [10]:
####################################################################################################
#  train and test the Kohonen Network with the new datasets
model = Kohonen(3)
model.train(x_train_PCA,y_train,x_test_PCA,y_test)
prediction_PCA =model.predict(x_test_PCA)
print(prediction_PCA)
print(y_test)
evaluator(y_test,prediction_PCA)

####################################################################################################

[0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2]
[0 0 0 0 0 2 1 0 2 1 1 0 1 1 2 0 1 2 2 0 2 2 2 1 0 2 2 1 1 1]
Confusion Matrix
[10, 9]
[1, 10]
Accuracy: 0.6666666666666666
Precision: 0.5263157894736842
Recall: 0.5263157894736842
F1 Score: 0.5263157894736842
